In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_covtype


In [0]:

from sklearn.neural_network import MLPClassifier

X, y = fetch_covtype(return_X_y=True)

In [0]:
X_10 = X[:,:10]

In [0]:
# Split the data into 90% training and 10% testing
from sklearn.model_selection import train_test_split
# The 10% testing data obtained during this split will be take as our entire database.
# This is because the original dataset is too big.
X10_train, X10_test, y10_train, y10_test = train_test_split(X_10, y, test_size=0.1,
stratify=y, random_state=42)

In [0]:
X = X10_test
y = y10_test

# Split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
# While splitting, make an unbiased splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
stratify=y, random_state=42)

In [0]:
# Feature scaling using Standardization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)

X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

mlpClassifier = MLPClassifier(solver='lbfgs', alpha=1e-5,
hidden_layer_sizes=(50, 25), random_state=1)

In [13]:

mlpClassifier.fit(X_train_std, y_train)


score = mlpClassifier.score(X_test_std, y_test)
print(score)


0.7513983306083813


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


## **USING KERAS MLP**

In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
# Setting the random seeds for repeatability
tf.set_random_seed(42)
np.random.seed(42)

# When return_X_y = True, the load function
# return data and target instead of Bunch object.
X, y = fetch_covtype(return_X_y=True)

In [0]:
# Reduce the number of attributes, consider only first 10 attributes.
X_10 = X[:,:10]

print(X_10.shape)
# (581012, 10)

# Split the data into 90% training and 10% testing
from sklearn.model_selection import train_test_split

# The 10% testing data obtained during this split will be take as our entire database.
# This is because the original dataset is too big.
X10_train, X10_test, y10_train, y10_test = train_test_split(X_10, y, test_size=0.1,
stratify=y, random_state=42)

print(X10_test.shape)
# (58102, 10)

# Handle only the modified 1% dataset. Split that into training and testing.
# X and y are updated with the downsized dataset
X = X10_test
y = y10_test

print(set(y))
# {1, 2, 3, 4, 5, 6, 7}
# The label should start from 0, but by default, the labels are from 1 to 7.
# Change them to the range 0 to 6 by subtracting 1 from the labels.
y = y-1
print(set(y))
# {0, 1, 2, 3, 4, 5, 6}

# Split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
# While splitting, make an unbiased splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
stratify=y, random_state=42)

# Feature scaling using Standardization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# Training the feature scaling parameters
sc.fit(X_train)


# Applying transformations to both training and testing set
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

(581012, 10)
(58102, 10)
{1, 2, 3, 4, 5, 6, 7}
{0, 1, 2, 3, 4, 5, 6}


In [0]:
# Create neural network using keras API
# Sequential() does linear stacking of layer
model_MLP = keras.models.Sequential()
# Hidden layer definitions
model_MLP.add(keras.layers.Dense(units=25, activation='relu',
input_shape= X_train.shape[1:]))
# Output layer definitions
model_MLP.add(keras.layers.Dense(units=7, activation='softmax'))

# Print the summary of network architecture
model_MLP.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 25)                275       
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 182       
Total params: 457
Trainable params: 457
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compile the network model with relevant configurations.
# loss, optimizer and metrics are three important configurations.
model_MLP.compile(loss='sparse_categorical_crossentropy',
optimizer='adam', metrics=['accuracy'])

In [0]:
model_MLP.fit(x=X_train_std, y=y_train, validation_split=0.1, epochs=5,batch_size=25)

Train on 41832 samples, validate on 4649 samples
Epoch 1/5
41832/41832 [==============================] - 3s 69us/sample - loss: 0.9108 - acc: 0.6388 - val_loss: 0.7232 - val_acc: 0.6991
Epoch 2/5
41832/41832 [==============================] - 3s 69us/sample - loss: 0.7162 - acc: 0.6953 - val_loss: 0.6855 - val_acc: 0.7107
Epoch 3/5
41832/41832 [==============================] - 3s 68us/sample - loss: 0.6915 - acc: 0.7010 - val_loss: 0.6700 - val_acc: 0.7199
Epoch 4/5
41832/41832 [==============================] - 3s 70us/sample - loss: 0.6803 - acc: 0.7052 - val_loss: 0.6597 - val_acc: 0.7206
Epoch 5/5
41832/41832 [==============================] - 3s 68us/sample - loss: 0.6733 - acc: 0.7070 - val_loss: 0.6521 - val_acc: 0.7208


In [0]:
test_loss, test_accuracy = model_MLP.evaluate(x=X_test_std, y=y_test)

11621/11621 [==============================] - 0s 24us/sample - loss: 0.6751 - acc: 0.7092


In [0]:
print(test_loss, test_accuracy)

0.675096897111683 0.7092333
